1. Estimate the channel capacity of plain old (wired) telephone service.

The Shannon capacity is $$ C = B\log_2(1+SNR) $$

The frequency spectrum of the human voice is from 300 to 3400 Hz and the typical SNR is 45dB according to https://www.jkaudio.com/article_03.htm 

In [3]:
import numpy as np

B = 3.1e3                       # Bandwidth in Hz   
SNR_dB = 45                     # signal to noise ratio in dB
SNR_linear = 10**(SNR_dB/10)    # linear conversion of SNR
C = B * np.log2(1 + SNR_linear) # Shannon capacity
print("The Shannon capacity is: ", C, "bits/s")

The Shannon capacity is:  46341.03834971382 bits/s


Using the point scatterer multipath propagation model from an earlier homework assignment,
(a) plot the SNR as a function of time for a moving receiver using spatial diversity with a simple selection algorithm, for isotropic radiator arrays of M = 1, 2, and 3 elements. 



(b) Plot the SNR CDFs with the analytical CDFs superimposed. 

(c) Plot the average SNR versus number of elements with the analytical result superimposed. 

(d) What happens if the array elements are very close together?